In [155]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data_torch
import torch.nn.functional as F
import random

In [156]:
# не обязательно
training_data = pd.read_csv('train.csv')
testing_data = pd.read_csv('test.csv')
display(training_data.head())
display(testing_data.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [157]:
class TitanicDataset(data_torch.Dataset):
    def __init__(self, filename, Train=True):
        self.dataframe = pd.read_csv(filename).copy()
        
        self.dataframe = self.dataframe.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
        mask = self.dataframe['Age'].isna()
        self.dataframe.loc[mask, 'Age'] = np.random.randint(10, 50, mask.sum())
        mask1 = self.dataframe['Embarked'].isna()
        self.dataframe.loc[mask1, 'Embarked'] = random.choice(self.dataframe['Embarked'].unique())
        self.dataframe['Sex'] = self.dataframe['Sex'].map({'male':1,'female':0})
        self.dataframe['Embarked'] = self.dataframe['Embarked'].map({'S':0,'C':1, 'Q':2})
        
        self.Train = Train
        
    def __len__(self):
        return self.dataframe.shape[0]
    
    def __getitem__(self, idx):
        if(self.Train):
            survived = self.dataframe['Survived']
            survived = np.array(survived)[idx]
        
        features = pd.DataFrame(columns=('Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'))
        features.loc[idx] = [ 
            self.dataframe.loc[idx, 'Pclass'],
            self.dataframe.loc[idx, 'Sex'],
            self.dataframe.loc[idx, 'Age'],
            self.dataframe.loc[idx, 'SibSp'],
            self.dataframe.loc[idx, 'Parch'],
            self.dataframe.loc[idx, 'Fare'],
            self.dataframe.loc[idx, 'Embarked']
        ]
        features = np.array(features)
        
        if(self.Train):
            return features, survived
        else:
            return features
          
    def infoo(self):
        return self.dataframe.info()

In [158]:
training_dataset = TitanicDataset('train.csv')
testing_dataset = TitanicDataset('test.csv', Train=False)

In [159]:
# training_dataset.__len__()

In [160]:
training_dataset.__getitem__(5)

(array([[ 3.    ,  1.    , 32.    ,  0.    ,  0.    ,  8.4583,  2.    ]]), 0)

In [161]:
training_dataset.infoo()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


In [162]:
dataloder = data_torch.DataLoader(training_dataset, shuffle=True, batch_size=100)
train_example_iterator = iter(dataloder)
print(next(train_example_iterator))

[tensor([[[  3.0000,   1.0000,  49.0000,   0.0000,   0.0000,   0.0000,   0.0000]],

        [[  3.0000,   1.0000,  29.0000,   0.0000,   0.0000,   7.8958,   1.0000]],

        [[  3.0000,   0.0000,   3.0000,   3.0000,   1.0000,  21.0750,   0.0000]],

        [[  1.0000,   0.0000,  35.0000,   1.0000,   0.0000,  83.4750,   0.0000]],

        [[  1.0000,   1.0000,  61.0000,   0.0000,   0.0000,  33.5000,   0.0000]],

        [[  3.0000,   1.0000,  25.0000,   0.0000,   0.0000,   7.2250,   1.0000]],

        [[  3.0000,   1.0000,  70.5000,   0.0000,   0.0000,   7.7500,   2.0000]],

        [[  3.0000,   1.0000,  19.0000,   0.0000,   0.0000,  14.5000,   0.0000]],

        [[  3.0000,   1.0000,  19.0000,   0.0000,   0.0000,   7.7500,   2.0000]],

        [[  3.0000,   1.0000,  40.5000,   0.0000,   0.0000,   7.7500,   2.0000]],

        [[  3.0000,   1.0000,  45.0000,   0.0000,   0.0000,   6.9750,   0.0000]],

        [[  1.0000,   1.0000,  70.0000,   1.0000,   1.0000,  71.0000,   0.0000]],

   

In [166]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dl1 = nn.Linear(7, 10)
        self.dl2 = nn.Linear(10, 12)
        self.dl3 = nn.Linear(12, 1)
        self.sigmoid = nn.Sigmoid()
            
    def forward(self, x):
        x = F.relu(self.dl1(x))
        x = F.relu(self.dl2(x))
        x = self.sigmoid(self.dl3(x))
        return x

In [169]:
net = Net()

In [168]:
inpu = torch.randn(1,7)
print(inpu)
out = net(inpu)
print(out)

tensor([[ 0.1153,  0.6383, -2.3063,  0.7041,  1.7621,  0.3635, -0.2165]])
tensor([[0.4256]], grad_fn=<SigmoidBackward0>)
